In [1]:
!nvidia-smi

Thu Aug 12 12:55:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:09:00.0 Off |                  Off |
| 63%   79C    P2   110W / 300W |   1281MiB / 48682MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
batch = 1024
n_epochs = 100
opt = "adam"
init_lr = 1e-3
use_amp = True

In [3]:
import argparse
import os
import shutil
import time
import pandas as pd
import network.resnet_orig as resnet
import matplotlib.pyplot as plt

import time
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

model_names = sorted(name for name in resnet.__dict__
    if name.islower() and not name.startswith("__")
                     and name.startswith("resnet")
                     and callable(resnet.__dict__[name]))

DATA_DIR = "train"

print_freq = 50

mixup = False

In [4]:
train_df = pd.read_csv("trainLabels.csv")
train_df.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['label_enc'] = le.fit_transform(train_df['label'])
train_df.head()

,id,label,label_enc
0,1,frog,6
1,2,truck,9
2,3,truck,9
3,4,deer,4
4,5,automobile,1


In [6]:
# 5-flod
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

train_df["fold"] = -1
for i, (train_index, test_index) in enumerate(skf.split(train_df.id, train_df.label_enc)):
    train_df.loc[test_index, 'fold'] = i
train_df.head()

,id,label,label_enc,fold
0,1,frog,6,1
1,2,truck,9,1
2,3,truck,9,2
3,4,deer,4,1
4,5,automobile,1,2


# prepare dataset

In [7]:
mixup = False

class cifarDataset(Dataset):
    def __init__(self,
                 df,
                 rand=False,
                 transform=None,
                 test=False
                ):

        self.df = df.reset_index(drop=True)
        self.rand = rand
        self.transform = transform
        self.test = test

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.id
        
        images = cv2.imread(os.path.join(DATA_DIR, str(img_id)+".png"))
        
        # Load labels
        label = row.label_enc
        
        # aug
        if self.transform is not None:
            images = self.transform(image=images)['image']
        
        # Mixup part
        """
        rd = torch.rand(1)
        label2 = label
        gamma = np.array(np.ones(1)).astype(np.float32)[0]
        if mixup and rd < 0.3 and self.transform is not None and not self.test:
            mix_idx = np.random.random_integers(0, len(self.df))
            row2 = self.df.iloc[mix_idx]
            img_id2 = row2.id
            images2 = cv2.imread(os.path.join(DATA_DIR, str(img_id2)+".png"))
            
            if self.transform is not None:
                images2 = self.transform(image=images2)['image']
            
            # blend image
            gamma = np.array(np.random.beta(1,1)).astype(np.float32)
            images = ((images*gamma + images2*(1-gamma))).astype(np.uint8)
            # blend labels
            label2 = row2.label_enc
        """
              
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)
        
        label = label.astype(np.float32)
        #label2 = label2.astype(np.float32)
        return torch.tensor(images), torch.tensor(label)

In [8]:
import albumentations as A
import albumentations

imsize = 32
transforms_train = albumentations.Compose([
    albumentations.ShiftScaleRotate(scale_limit=0.3, rotate_limit=180,p=0.5),
    A.OneOf([
        A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                             val_shift_limit=0.2, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, 
                                   contrast_limit=0.2, p=0.5),
    ],p=0.9),
    A.Cutout(num_holes=12, max_h_size=4, max_w_size=4, fill_value=0, p=0.5),
    albumentations.Rotate(p=0.5),
    albumentations.Transpose(p=0.5),
    #albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),   
    albumentations.Resize(imsize, imsize, p=1.0), 
])
transforms_val = albumentations.Compose([albumentations.Resize(imsize, imsize, p=1.0)])

/home/ken/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:690: FutureWarning: This class has been deprecated. Please use CoarseDropout
  FutureWarning,


show imgs

In [9]:
dataset_show = cifarDataset(train_df, transform=transforms_train)
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
for i in range(3):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = np.random.randint(0, len(dataset_show))
        img, label = dataset_show[idx]
        img = img.flip(0) #BGR2RGB
        axarr[p].imshow(img.transpose(0,1).transpose(1,2))
        axarr[p].set_title(str(label))

# Trainer

In [10]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [11]:
def train(train_loader, model, criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    lambda_alpha = 0.00002
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        target = target.long().cuda()
        input_var = input.cuda()
        target_var = target

        if not mixup:
          alpha = 0
        else:
          alpha = 1
        
        with torch.cuda.amp.autocast(enabled=use_amp):
            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

        # L2 regularization
        l2_alpha = 0.0
        for name, param in model.named_parameters():
            if "alpha" in name:
                l2_alpha += torch.pow(param, 2)
        loss += lambda_alpha * l2_alpha

        # compute gradient and do SGD step          
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      epoch, i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
    return losses.avg

def validate(val_loader, model, criterion):
    """
    Run evaluation
    """
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.long().cuda()
            input_var = input.cuda()
            target_var = target.cuda()

            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(val_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'
          .format(top1=top1))
    for name, param in model.named_parameters():
        if "alpha" in name:
            print(name, param.item())
    return top1.avg, losses.avg

def save_checkpoint(state, filename='checkpoint.pth'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# Train loop

In [12]:
train_dataset = cifarDataset(train_df[train_df.fold!=0], transform=transforms_train)
val_dataset = cifarDataset(train_df[train_df.fold==0], transform=transforms_val, test=True)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch, shuffle=True,
        num_workers=8, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch, shuffle=False,
    num_workers=8, pin_memory=True)

In [ ]:
for net in ["resnet34", "resnet50"]:
    for K in [3,4,6,8]:
        model = resnet.__dict__[net](K)
        model = model.cuda()

        import wandb
        watermark = "{}_pact{}bit".format(net, K)
        wandb.init(project="pact_forpaper",
                    name=watermark)

        # define loss function (criterion) and pptimizer
        criterion = nn.CrossEntropyLoss().cuda()

        def mixup_criterion(criterion, pred, y_a, y_b, lam):
            return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

        if opt=="adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
        
        scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-1)

        best_prec1 = 0
        os.makedirs("models", exist_ok=True)

        for epoch in range(n_epochs):
            # train for one epoch
            start = time.time()
            print('current lr {:.5e}'.format(optimizer.param_groups[0]['lr']))
            tloss = train(train_loader, model, criterion, optimizer, epoch)
            lr_scheduler.step()

            # evaluate on validation set
            prec1, valloss = validate(val_loader, model, criterion)
            
            end = time.time() - start
            # wandb
            wandb.log({'epoch': epoch, "prec":prec1, "train_loss": tloss, 'val_loss': valloss, "epoch_time":end, "lr": optimizer.param_groups[0]["lr"],})

            # remember best prec@1 and save checkpoint
            is_best = prec1 > best_prec1
            best_prec1 = max(prec1, best_prec1)

            print("Best prec1 : ", best_prec1)
            if is_best:
                torch.save(model.state_dict(), os.path.join(f'models/{net}_quant{K}.pth'))

bit width: 3


wandb: Currently logged in as: arutema47 (use `wandb login --relogin` to force relogin)


current lr 1.00000e-03
Epoch: [0][0/40]	Time 2.675 (2.675)	Data 1.630 (1.630)	Loss 7.9151 (7.9151)	Prec@1 10.254 (10.254)
Test: [0/10]	Time 0.708 (0.708)	Loss 3.7535 (3.7535)	Prec@1 15.430 (15.430)
 * Prec@1 14.070
alpha1 9.961016654968262
layer1.0.alpha1 9.961016654968262
layer1.0.alpha2 9.963092803955078
layer1.1.alpha1 9.961016654968262
layer1.1.alpha2 9.967028617858887
layer1.2.alpha1 9.961016654968262
layer1.2.alpha2 9.986981391906738
layer2.0.alpha1 9.961016654968262
layer2.0.alpha2 9.961400985717773
layer2.1.alpha1 9.961016654968262
layer2.1.alpha2 9.964095115661621
layer2.2.alpha1 9.961016654968262
layer2.2.alpha2 9.974077224731445
layer2.3.alpha1 9.961016654968262
layer2.3.alpha2 9.991313934326172
layer3.0.alpha1 9.961016654968262
layer3.0.alpha2 9.961020469665527
layer3.1.alpha1 9.961016654968262
layer3.1.alpha2 9.961971282958984
layer3.2.alpha1 9.961016654968262
layer3.2.alpha2 9.967643737792969
layer3.3.alpha1 9.961016654968262
layer3.3.alpha2 9.980507850646973
layer3.4.alp

Test: [0/10]	Time 0.720 (0.720)	Loss 1.7971 (1.7971)	Prec@1 37.891 (37.891)
 * Prec@1 39.480
alpha1 9.723559379577637
layer1.0.alpha1 9.72354793548584
layer1.0.alpha2 9.728409767150879
layer1.1.alpha1 9.723597526550293
layer1.1.alpha2 9.741388320922852
layer1.2.alpha1 9.723562240600586
layer1.2.alpha2 9.866499900817871
layer2.0.alpha1 9.723562240600586
layer2.0.alpha2 9.724542617797852
layer2.1.alpha1 9.723562240600586
layer2.1.alpha2 9.732161521911621
layer2.2.alpha1 9.723562240600586
layer2.2.alpha2 9.765347480773926
layer2.3.alpha1 9.723562240600586
layer2.3.alpha2 9.885980606079102
layer3.0.alpha1 9.723562240600586
layer3.0.alpha2 9.72352123260498
layer3.1.alpha1 9.723562240600586
layer3.1.alpha2 9.725334167480469
layer3.2.alpha1 9.723562240600586
layer3.2.alpha2 9.740091323852539
layer3.3.alpha1 9.723562240600586
layer3.3.alpha2 9.811277389526367
layer3.4.alpha1 9.723562240600586
layer3.4.alpha2 9.889022827148438
layer3.5.alpha1 9.723562240600586
layer3.5.alpha2 9.982885360717773


current lr 9.58054e-04
Epoch: [13][0/40]	Time 1.581 (1.581)	Data 1.219 (1.219)	Loss 1.7720 (1.7720)	Prec@1 38.867 (38.867)
Test: [0/10]	Time 0.802 (0.802)	Loss 1.5923 (1.5923)	Prec@1 44.629 (44.629)
 * Prec@1 43.270
alpha1 9.455976486206055
layer1.0.alpha1 9.456046104431152
layer1.0.alpha2 9.46408748626709
layer1.1.alpha1 9.455964088439941
layer1.1.alpha2 9.48037052154541
layer1.2.alpha1 9.455748558044434
layer1.2.alpha2 9.67140007019043
layer2.0.alpha1 9.455869674682617
layer2.0.alpha2 9.45744800567627
layer2.1.alpha1 9.455892562866211
layer2.1.alpha2 9.466222763061523
layer2.2.alpha1 9.455869674682617
layer2.2.alpha2 9.51450252532959
layer2.3.alpha1 9.455869674682617
layer2.3.alpha2 9.721048355102539
layer3.0.alpha1 9.455869674682617
layer3.0.alpha2 9.45604419708252
layer3.1.alpha1 9.455869674682617
layer3.1.alpha2 9.460098266601562
layer3.2.alpha1 9.455869674682617
layer3.2.alpha2 9.481864929199219
layer3.3.alpha1 9.455869674682617
layer3.3.alpha2 9.604792594909668
layer3.4.alpha1 9

...

current lr 1.00000e-03
Epoch: [0][0/40]	Time 2.027 (2.027)	Data 1.660 (1.660)	Loss 6.7467 (6.7467)	Prec@1 8.301 (8.301)
Test: [0/10]	Time 0.801 (0.801)	Loss 2.7800 (2.7800)	Prec@1 21.094 (21.094)
 * Prec@1 22.790
alpha1 9.961015701293945
layer1.0.alpha1 9.96114730834961
layer1.0.alpha2 9.96588134765625
layer1.1.alpha1 9.961121559143066
layer1.1.alpha2 9.981550216674805
layer1.2.alpha1 9.960795402526855
layer1.2.alpha2 9.991941452026367
layer2.0.alpha1 9.961231231689453
layer2.0.alpha2 9.96427059173584
layer2.1.alpha1 9.960983276367188
layer2.1.alpha2 9.976947784423828
layer2.2.alpha1 9.960832595825195
layer2.2.alpha2 9.98831558227539
layer2.3.alpha1 9.961112022399902
layer2.3.alpha2 9.991268157958984
layer3.0.alpha1 9.961091041564941
layer3.0.alpha2 9.990835189819336
layer3.1.alpha1 9.96139144897461
layer3.1.alpha2 10.002418518066406
layer3.2.alpha1 9.96101188659668
layer3.2.alpha2 9.994424819946289
layer3.3.alpha1 9.961004257202148
layer3.3.alpha2 9.989961624145508
layer3.4.alpha1 9.9

Test: [0/10]	Time 0.920 (0.920)	Loss 1.6527 (1.6527)	Prec@1 41.211 (41.211)
 * Prec@1 43.070
alpha1 9.723554611206055
layer1.0.alpha1 9.723913192749023
layer1.0.alpha2 9.730278968811035
layer1.1.alpha1 9.72390365600586
layer1.1.alpha2 9.769729614257812
layer1.2.alpha1 9.722896575927734
layer1.2.alpha2 9.937151908874512
layer2.0.alpha1 9.724150657653809
layer2.0.alpha2 9.724981307983398
layer2.1.alpha1 9.7234468460083
layer2.1.alpha2 9.786700248718262
layer2.2.alpha1 9.722929000854492
layer2.2.alpha2 9.908984184265137
layer2.3.alpha1 9.72380256652832
layer2.3.alpha2 9.961801528930664
layer3.0.alpha1 9.723723411560059
layer3.0.alpha2 9.817399978637695
layer3.1.alpha1 9.724531173706055
layer3.1.alpha2 9.932223320007324
layer3.2.alpha1 9.723553657531738
layer3.2.alpha2 9.890010833740234
layer3.3.alpha1 9.723538398742676
layer3.3.alpha2 9.920958518981934
layer3.4.alpha1 9.723562240600586
layer3.4.alpha2 9.937725067138672
layer3.5.alpha1 9.723562240600586
layer3.5.alpha2 9.976652145385742
la

 * Prec@1 49.650
alpha1 9.493269920349121
layer1.0.alpha1 9.493714332580566
layer1.0.alpha2 9.499797821044922
layer1.1.alpha1 9.493696212768555
layer1.1.alpha2 9.550878524780273
layer1.2.alpha1 9.492443084716797
layer1.2.alpha2 9.845342636108398
layer2.0.alpha1 9.493968963623047
layer2.0.alpha2 9.49547004699707
layer2.1.alpha1 9.493146896362305
layer2.1.alpha2 9.576125144958496
layer2.2.alpha1 9.492539405822754
layer2.2.alpha2 9.785014152526855
layer2.3.alpha1 9.493571281433105
layer2.3.alpha2 9.912915229797363
layer3.0.alpha1 9.493474006652832
layer3.0.alpha2 9.612960815429688
layer3.1.alpha1 9.49441909790039
layer3.1.alpha2 9.823243141174316
layer3.2.alpha1 9.493277549743652
layer3.2.alpha2 9.739104270935059
layer3.3.alpha1 9.493258476257324
layer3.3.alpha2 9.81900405883789
layer3.4.alpha1 9.493287086486816
layer3.4.alpha2 9.85466194152832
layer3.5.alpha1 9.493287086486816
layer3.5.alpha2 9.94017219543457
layer4.0.alpha1 9.493287086486816
layer4.0.alpha2 9.667092323303223
layer4.1.al

current lr 9.11838e-04
Epoch: [19][0/40]	Time 1.527 (1.527)	Data 1.162 (1.162)	Loss 1.3234 (1.3234)	Prec@1 54.004 (54.004)
Test: [0/10]	Time 0.780 (0.780)	Loss 1.1520 (1.1520)	Prec@1 60.059 (60.059)
 * Prec@1 58.620
alpha1 9.238635063171387
layer1.0.alpha1 9.23902702331543
layer1.0.alpha2 9.24680233001709
layer1.1.alpha1 9.239175796508789
layer1.1.alpha2 9.30385684967041
layer1.2.alpha1 9.23762321472168
layer1.2.alpha2 9.713537216186523
layer2.0.alpha1 9.23949146270752
layer2.0.alpha2 9.240575790405273
layer2.1.alpha1 9.238383293151855
layer2.1.alpha2 9.330538749694824
layer2.2.alpha1 9.2377290725708
layer2.2.alpha2 9.62087631225586
layer2.3.alpha1 9.238855361938477
layer2.3.alpha2 9.847076416015625
layer3.0.alpha1 9.238748550415039
layer3.0.alpha2 9.37427806854248
layer3.1.alpha1 9.239771842956543
layer3.1.alpha2 9.672591209411621
layer3.2.alpha1 9.238539695739746
layer3.2.alpha2 9.547311782836914
layer3.3.alpha1 9.238519668579102
layer3.3.alpha2 9.684450149536133
layer3.4.alpha1 9.23

Test: [0/10]	Time 0.800 (0.800)	Loss 0.9585 (0.9585)	Prec@1 67.578 (67.578)
 * Prec@1 65.850
alpha1 9.035962104797363
layer1.0.alpha1 9.0357666015625
layer1.0.alpha2 9.045771598815918
layer1.1.alpha1 9.036423683166504
layer1.1.alpha2 9.105894088745117
layer1.2.alpha1 9.03487491607666
layer1.2.alpha2 9.594294548034668
layer2.0.alpha1 9.036930084228516
layer2.0.alpha2 9.039255142211914
layer2.1.alpha1 9.035591125488281
layer2.1.alpha2 9.13386344909668
layer2.2.alpha1 9.03488826751709
layer2.2.alpha2 9.476937294006348
layer2.3.alpha1 9.036057472229004
layer2.3.alpha2 9.793548583984375
layer3.0.alpha1 9.03594970703125
layer3.0.alpha2 9.17958927154541
layer3.1.alpha1 9.037005424499512
layer3.1.alpha2 9.534615516662598
layer3.2.alpha1 9.03573226928711
layer3.2.alpha2 9.396429061889648
layer3.3.alpha1 9.035714149475098
layer3.3.alpha2 9.571672439575195
layer3.4.alpha1 9.035744667053223
layer3.4.alpha2 9.662420272827148
layer3.5.alpha1 9.035744667053223
layer3.5.alpha2 9.833062171936035
layer4